# Before running this notebook, you need to finish the "preprocess_waymo" notebook first

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd

import sys
sys.path.append('../src/')

import utils.utils as utils
import utils.dataloader_utils as dataloader_utils
import utils.train_utils as train_utils

import json
from tqdm import tqdm

%load_ext autoreload
%autoreload 2

In [ ]:
# You need GPU to run this notebook
tf.config.experimental.list_physical_devices('GPU')

## Step 1: Load Data

In [ ]:
BATCH_SIZE = 1024
num_points_in_one_traj = 11

In [ ]:
with open("data/ego/ego_trajs_" + str(num_points_in_one_traj) + "_json/ego_trajs_outlier_indicies.json", "r") as read_file:
    idx_invalid_idx = set(json.load(read_file))

list_dataset = dataloader_utils.generate_file_list_dataset('data/ego/ego_trajs_json/', idx_invalid_idx)
start_idx_dataset = dataloader_utils.generate_start_indicies_dataset("data/ego/ego_trajs_" + str(num_points_in_one_traj) + "_json/ego_trajs_start_point_indicies.json", idx_invalid_idx)
combined_dataset = tf.data.Dataset.zip((list_dataset, start_idx_dataset))

dataProcessor = dataloader_utils.DataProcessor(BATCH_SIZE, combined_dataset, num_points_in_one_traj, traj_type = 'ego_traj')
dataProcessor.load_process(shuffle = True)

print(dataProcessor.loaded_dataset.__len__())

## Step 2: Empirical Bayes Analysis for Ego Trajectory

In [8]:
# longer trajectories need more epochs
EPOCHS = 50 

# longer trajectories need lower lr, consider using https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/schedules/PiecewiseConstantDecay
lr = 5e-3 

In [ ]:
losses = []
best_epoch_losses = []
best_epochs = []
bic_scores = []
aic_scores = []
A_list, B_list = [], []
optimizers_ser = []
log_root_dir = 'logs/gradient_tape/ego/ego_xy' + str(num_points_in_one_traj)
t_scale_factor = (num_points_in_one_traj-1) / 10 # The time duration of one trajectory, for scaling time to interval (0,1)
degrees = np.linspace(1, 8, 8, dtype=np.int16) # analyse polynomials from degree 1 to 8

for i_d, deg in enumerate(degrees):
    print('Analysing Deg ',deg)

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)   
    optimizers_ser.append(tf.keras.optimizers.serialize(optimizer))
    
    # Initialize all trainable variables
    A = tf.Variable(np.random.randn(2*(deg), 2*(deg)) * 1e-1 , dtype=tf.float64, name='alpha') # Model uncertainty. Note: 0-th parameter (start position) is not interested
    B_diag = tf.Variable(np.random.randn(1) * 1e-1, dtype=tf.float64, name='beta_diag') # Log of Observation uncertainty
    B_by_diag =  tf.Variable(np.random.randn(1) * 1e-1, dtype=tf.float64, name='beta_by_diag')

    
    train_log_dir = log_root_dir + '/deg_' + str(deg)
    train_summary_writer = tf.summary.create_file_writer(train_log_dir)  
   
    model_losses, best_epoch_loss, best_epoch, best_alpha, best_beta_diag, best_beta_by_diag = train_utils.train_ego(alpha=A, beta_diag=B_diag, beta_by_diag=B_by_diag, t_scale_factor = t_scale_factor, degree = deg,
                                                                                                                     opti=optimizer, epochs = EPOCHS, data_loader=dataProcessor.loaded_dataset, tf_summary_writer = train_summary_writer, 
                                                                                                                     verbose = True, early_stop=False)
            
    # Add model loss
    losses.append(model_losses)
    best_epoch_losses.append([best_epoch_loss])
    
    # store the best epoch
    best_epochs.append(best_epoch)
    
    # Compute the AIC and BIC score
    aic_score, bic_score = utils.compute_AIC_BIC(nll = best_epoch_loss, deg = deg, num_points = num_points_in_one_traj, dof_in_ob = 2)

    bic_scores.append(bic_score)
    aic_scores.append(aic_score)
    
    # Compute the model uncertainty, A_unscaled = np.linalg.inv(scale_mat) @ A_scaled
    A_scale_mat = utils.polyBasisScale(t_scale_factor, deg)
    A_scale_mat = A_scale_mat[1:, 1:]
    A_est = np.linalg.inv(np.kron(np.eye(2), A_scale_mat)) @ best_alpha.numpy()
    A_est = A_est @ A_est.T
    A_list.append(A_est)
    
    # Compute the observation uncertainty, B_cov = tf.eye(num_points_in_one_traj) * tf.math.softplus(B)
    B_est = {'B_diag': (tf.math.softplus(best_beta_diag)).numpy(), 
             'B_by_diag': (tf.math.softplus(best_beta_diag) * tf.math.tanh(best_beta_by_diag)).numpy()}
    B_list.append(B_est)
    print("Degree: {}, NLL: {}, BIC: {}, AIC: {}".format(deg, model_losses[-1], bic_score, aic_score))

In [ ]:
result = utils.calculate_result(degrees, bic_scores, aic_scores, A_list, B_list, best_epoch_losses, best_epochs, lr, optimizers_ser, EPOCHS, BATCH_SIZE)

In [ ]:
utils.save_result(folder_dir =log_root_dir, file_name='result_summary', result=result)
with open(log_root_dir + '/' + 'optimizers' + '.json', "w") as write_file:
    json.dump(optimizers_ser, write_file, cls=NumpyEncoder)